# Replacing the UK personal allowance with a universal basic income

The personal allowance is a feature of the UK tax system which designates a certain amount of individual income as non-taxable. This amount is currently set at £12,500, meaning the first £12,500 of an individual’s annual earnings are not taxed. Since it was [introduced in 1979 at £1,165](http://taxhistory.co.uk/Income%20Tax%20Allowances.htm) (equivalent to around [£6,028 today](https://www.bankofengland.co.uk/monetary-policy/inflation/inflation-calculator)), it has increased almost every year.

The following analysis explores repealing the personal allowance and using the resulting tax revenue to fund a universal basic income. The findings show that this would be an effective policy for addressing poverty, reducing the overall poverty rate by 29% and deep poverty by 46%. It is especially effective for reducing child poverty, which would fall a remarkable 51%. Lastly, it shows that each personal allowance reduction of £2,000 could yield around a 5% decrease in poverty.

#### Debate around the personal allowance


The personal allowance is [very popular in the UK](https://www.ipsos.com/ipsos-mori/en-uk/personal-allowances-rise-most-popular-conference-season-tax-pledges), as it allows people to retain a greater portion of their income. At current levels, the personal allowance lowers the average worker's tax liability by £1,600 per year.

Moreover, supporters of the personal allowance often characterize it [as a progressive anti-poverty measure](https://www.standard.co.uk/news/politics/libdems-to-let-1-3m-low-earners-avoid-paying-tax-8506838.html), since it allows the lowest income earners—those making under £12,500—to be free from any income tax at all, while it phases out for the highest earners making above £100,000. Many people support raising the personal allowance thinking that the higher it is, the greater the number of low income earners who fall below the threshold, allowing these individuals to retain more of their income. Thus, by this logic, it should serve as a mechanism for reducing poverty.

Though this sounds good on paper, personal allowance increases have been found not to be progressive in practice. There are two main reasons for this. Firstly, many of the UK’s poorest already fall below the personal allowance threshold, and raising the personal allowance does nothing for them. Rather, it exclusively helps higher income earners who make above the threshold and up to [£125,140](https://www.gov.uk/income-tax-rates/income-over-100000). Though some at the bottom of the income scale may benefit, the [overwhelming majority of the benefits do not go to them](https://leftfootforward.org/2013/03/the-10000-personal-tax-allowance-anything-but-progressive/) but to those more affluent.

The second reason has to do with the phaseout of Universal Credit and similar means-tested benefits. Since Universal Credit phases out with respect to post-tax income, any tax reduction also reduces Universal Credit payments. As [explained](https://www.politics.co.uk/opinion-former/press-release/2018/10/29/personal-allowance-increase-does-little-for-those-on-lowest-income/) by Victoria Todd, Head of the Low Income Tax Reform Group, about the 2019 changes: 

> “[Universal credit recipients] will not see the full tax gain of £130 from the increase in the personal allowance; instead, they will only gain overall by £48.10, as their Universal Credit award will be reduced by £81.90. However, those earning above £11,850 who receive tax credits will benefit from the full £130 because tax credits are based on gross income.”

Again, this means that higher earners benefit more than low-income earners from personal reform increases. In fact, according to the [Resolution Foundation](https://www.resolutionfoundation.org/app/uploads/2014/12/Missing-the-target1.pdf), the most recent increase of the personal allowance from £11,850 to £12,500.

One last criticism of the personal allowance is that its phase-out creates an extremely high marginal tax rate of around 62% for those earning between £100,000 and £125,000. A more detailed analysis of this can be found in [our report modeling a blank slate UBI in the UK](https://d3n8a8pro7vhmx.cloudfront.net/socialliberalforum/pages/3398/attachments/original/1620835589/SLF_UBI_Center_Report_FINAL.pdf?1620835589).

#### Proposals regarding personal allowance and UBI

These criticisms suggest that eliminating the personal allowance would only reduce the income of the UK’s poorest negligibly, while the more affluent would see a greater burden. This makes it a good candidate for helping fund a progressive UBI proposal.

Both UK parties who have released UBI proposals have found a role for personal allowance to help fund their programs. The Liberal Democrats' [discussion paper](https://d3n8a8pro7vhmx.cloudfront.net/libdems/pages/1811/attachments/original/1621669347/145_-_Universal_Basic_Income.docx_%281%29.pdf?1621669347) proposes reducing the Personal Allowance to £2,500 a year or higher (see our [analysis](https://www.ubicenter.org/lib-dem-policy-paper) of that plan). Meanwhile, the Green Party states that in their model, UBI will be taxable, but [“all income tax payers will have a tax-free allowance which is the equivalent to their Universal Basic Income amount”](https://www.greenparty.ie/wp-content/uploads/2018/07/Green-Party-Universal-Basic-Income-Policy.pdf); this means that, in practice, the UBI would not be taxed and the personal allowance would effectively be eliminated. Both parties also include UBI when means-testing.

In 2019, the New Economics Foundation [modeled](https://neweconomics.org/2019/03/nothing-personal) replacing the Personal Allowance with a Weekly National Allowance paid to adults and an increase in the Child Benefit, making the payments non-taxable but included in means tests, and found progressive results. Compass has also [modeled](https://www.compassonline.org.uk/wp-content/uploads/2019/03/Compass_BasicIncomeForAll_2019.pdf) replacing the personal allowance with a flat payment of £25 a week and found progressive results.

In the following analysis, we will explore the effect of a UBI funded exclusively through the elimination of a personal allowance. The UBI amount in this simulation is equal for everyone, regardless of age or disability, and does not count towards taxation or means-testing.

#### Our findings

Fully repealing the personal allowance could fund a UBI of £1,488 per person. This policy would, in general, serve as an income transfer from the richest 40% to the poorest 60%. 

In [796]:
# Setup
from openfisca_uk import Microsimulation
from openfisca_uk import *
from openfisca_core.model_api import Reform
from openfisca_uk.entities import Person, BenUnit, Household
from openfisca_core.model_api import *
from openfisca_uk.tools.general import *
from ubicenter import format_fig
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"
#change "browser" to "notebook" when working on non-ines computers

sim = Microsimulation(input_year=2020)

from openfisca_core import periods
def make_PA_reform(PA_amount):
    
    def update_PA_parameter(parameters):
        parameters.tax.income_tax.allowances.personal_allowance.amount.update(period=periods.period("year:2020:1"), value=PA_amount)
        return parameters
    
    class reform(Reform):
        def apply(self):
            self.modify_parameters(update_PA_parameter)
    
    sim_less_PA = Microsimulation(reform, input_year=2020)
    revenue = sim.calc("net_income").sum()
    revenue_diff = revenue - sim_less_PA.calc("net_income").sum()
    BI_amount = revenue_diff/(sim.calc("people").sum())
    
    class BI(Variable):
        value_type = float
        entity = Person
        label = u"UBI"
        definition_period = YEAR
        def formula(person, period, parameters):
            return(BI_amount)

        
    class gross_income(Variable):
        value_type = float
        entity = Person
        label = u"Gross income, including benefits"
        definition_period = YEAR

        def formula(person, period, parameters):
            COMPONENTS = [
                "employment_income",
                "pension_income",
                "self_employment_income",
                "property_income",
                "savings_interest_income",
                "dividend_income",
                "miscellaneous_income",
                "benefits",
                "BI"
            ]
            return add(person, period, COMPONENTS)   

    class basic_income(Reform):
        def apply(self):
            self.add_variable(BI)
            self.update_variable(gross_income)
            
    sim_BI = Microsimulation(reform, basic_income, input_year=2020)
    
    #return sim_BI
    return reform, basic_income

#sim_BI = make_PA_reform(0)


In [804]:
#Only run once 

reform_list = [make_PA_reform(i) for i in PA_amounts2]
sim_list = [Microsimulation(reform, input_year=2020) for reform in reform_list]

In [808]:
sim_BI = sim_list[0]

In [901]:
sim = Microsimulation(input_year=2020)

In [781]:
income_diff = sim_BI.calc("household_net_income") - sim.calc("household_net_income")
income = sim.calc("equiv_household_net_income")

In [805]:
def find_income_diff_deciles(sim_BI):
    income_diff = sim_BI.calc("household_net_income") - sim.calc("household_net_income")
    income_diff_decile = income_diff.groupby(income.decile_rank()).mean().round()
    return income_diff_decile

PA_amounts = [12500, 11500, 10500, 9500, 8500, 7500, 6500, 5500, 4500, 3500, 2500, 1500, 500, 0]
PA_amounts2 = [0, 500, 1500, 2500, 3500, 4500, 5500, 6500, 7500, 8500, 9500, 10500, 11500, 12500]

In [806]:
#Decile impact (scrollable)
df_list = []
sim_num = 0
for i in PA_amounts2:
    df = pd.DataFrame({"decile": deciles[1:], "Change in income": find_income_diff_deciles(sim_list[sim_num]), "PA amount": i})
    df_list.append(df)
    sim_num += 1
    
final_df = pd.concat(df_list)
format_fig(px.bar(final_df, x="decile", y="Change in income", animation_frame="PA amount", range_y=[-2500,2500],
                 hover_data={"PA amount": False, "decile": False}).update_layout(
    title_text='Effect of replacing personal allowance with UBI on net income by decile',
    xaxis_title ="Equivalized household net income decile",
    yaxis_title ="Change in household net income",
    showlegend= False,
    yaxis_tickprefix="£",
    #hovermode="y unified",
).update_traces(marker_color='#1976D2'))

In [633]:
#Decile impact (not scrollable)
chart1 = format_fig(px.bar(income_diff.groupby(income.decile_rank()).mean().round(), labels={"index": "decile", "value":"change in income"},
                          hover_data={"variable": False}).update_layout(
    title_text='Average change in equivalized household net income for each income decile',
    xaxis_title ="Household income decile",
    yaxis_title ="Change in equivalized household net income",
    showlegend= False,
    yaxis_tickprefix="£",
))

Median household income would increase by about £564, and this number is over five times greater for those living in poverty.

In [810]:
#Median difference in household net income for different poverty groups (not scrollable)
effect = sim_BI.calc("household_net_income", map_to="person") - sim.calc("household_net_income", map_to="person")
isDeepPoor = sim.calc("in_deep_poverty_bhc", map_to ="person")
isPoor = sim.calc("in_poverty_bhc", map_to = "person") & ~isDeepPoor
#isPoor excludes those in deep poverty
overall = sim_BI.calc("household_net_income").median() - sim.calc("household_net_income").median()

chart2 = format_fig(px.bar(x=["Deep poverty", "In poverty, but not deep", "Not in poverty", "Overall"], 
                           y=[effect[isDeepPoor].median().round(), effect[isPoor].median().round(), effect[~isPoor].median().round(), overall],
                          labels={"x": "poverty status", "y":"gain in income"}).update_layout(
    title_text='Median gain in household net income for different poverty groups',
    xaxis_title ="Group",
    yaxis_title ="Median gain in household net income",
    yaxis_tickprefix="£",
))

In [501]:
#Calculate mean difference in household net income for different poverty groups
effect = sim_BI.calc("household_net_income", map_to="person") - sim.calc("household_net_income", map_to="person")

def find_DP(sim_BI):
    effect = sim_BI.calc("household_net_income", map_to="person") - sim.calc("household_net_income", map_to="person")
    isDeepPoor = sim.calc("in_deep_poverty_bhc", map_to ="person")
    return effect[isDeepPoor].median().round()

def find_pov(sim_BI):
    effect = sim_BI.calc("household_net_income", map_to="person") - sim.calc("household_net_income", map_to="person")
    isPoor = sim.calc("in_poverty_bhc", map_to = "person") & ~isDeepPoor
    return effect[isPoor].median().round()

#Unused
def find_not_pov(sim_BI):
    effect = sim_BI.calc("household_net_income", map_to="person") - sim.calc("household_net_income", map_to="person")
    return effect[~isPoor].median().round()

def find_overall(sim_BI):
    effect = sim_BI.calc("household_net_income", map_to="person") - sim.calc("household_net_income", map_to="person")
    return sim_BI.calc("household_net_income").median().round() - sim.calc("household_net_income").median().round()

In [811]:
#Median difference in household net income for different poverty groups (scrollable)

df_list = []
sim_num = 0
poverty_types = ["Deep poverty", "Poverty, not deep", "Overall"]

#df = pd.DataFrame({"deciles": deciles[1:], "changes": find_income_diff_deciles(sim_list[sim_num]), "PA amount": i})
for i in PA_amounts2:
    changes = pd.Series([find_DP(sim_list[sim_num]), find_pov(sim_list[sim_num]), find_overall(sim_list[sim_num])])
    df = pd.DataFrame({"Poverty group": poverty_types, "Income gain": changes, "PA amount": i})
    df_list.append(df)
    sim_num += 1


final_df = pd.concat(df_list)
final_df
format_fig(px.bar(final_df, x="Poverty group", y="Income gain", animation_frame="PA amount", range_y=[0,3500],
                 hover_data={"PA amount": False, "Poverty group": False}).update_layout(
    title_text='Median gain in household net income for different poverty groups',
    xaxis_title ="",
    yaxis_title ="Median gain in household net income",
    yaxis_tickprefix="£",
    ).update_traces(marker_color='#1976D2'))


The policy would reduce poverty by 29% and deep poverty (people living at under half the poverty line) by 46%.

In [414]:
#Reductions in poverty rates (not scrollable)

import plotly.graph_objects as go

poverty_before = sim.calc("in_poverty_bhc", map_to="person").sum()/sim.calc("people").sum() * 100
poverty_now = sim_BI.calc("in_poverty_bhc", map_to="person").sum()/sim.calc("people").sum() * 100

deep_poverty_before = sim.calc("in_deep_poverty_bhc", map_to="person").sum()/sim.calc("people").sum() * 100
deep_poverty_now = sim_BI.calc("in_deep_poverty_bhc", map_to="person").sum()/sim.calc("people").sum() * 100

poverty_types=['Poverty', "Deep poverty"]

chart3 = format_fig(go.Figure(data=[
    go.Bar(name='Before reform', x=poverty_types, y=[poverty_before.round(), deep_poverty_before.round()], marker_color='gray'),
    go.Bar(name='After reform', x=poverty_types, y=[poverty_now.round(), deep_poverty_now.round()], marker_color='rgb(26, 118, 255)')
]).update_layout(
    barmode='group', 
    title_text='Poverty rates before and after reform', 
    xaxis_title ="Poverty type",
    yaxis_title ="Rate",
    yaxis_ticksuffix="%", ))

In [698]:
#Reductions in poverty rates (scrollable)
poverty_types= pd.Series(['Poverty', "Deep poverty"])
df_list = []
sim_num = 0
def find_pov_change(sim_BI):
    #return sim_BI.calc("in_poverty_bhc", map_to="person").sum()/sim.calc("people").sum() * 100
    return (sim.calc("in_poverty_bhc", map_to="person").sum() - sim_BI.calc("in_poverty_bhc", map_to="person").sum())/sim.calc("in_poverty_bhc", map_to="person").sum() * -100 

def find_deep_pov_change(sim_BI):
    #return sim_BI.calc("in_deep_poverty_bhc", map_to="person").sum()/sim.calc("people").sum() * 100
    return (sim.calc("in_deep_poverty_bhc", map_to="person").sum() - sim_BI.calc("in_deep_poverty_bhc", map_to="person").sum())/sim.calc("in_deep_poverty_bhc", map_to="person").sum() * -100 


for i in PA_amounts2:
#if change to PA_amounts, sim_list needs to be parsed backwards (sim_list.length - sim_num)
    changes = pd.Series([find_pov_change(sim_list[sim_num]), find_deep_pov_change(sim_list[sim_num])])
    df = pd.DataFrame({"Poverty type": poverty_types, "Poverty rate": changes, "PA amount": i})
    df_list.append(df)
    sim_num += 1
    
final_df = pd.concat(df_list)
format_fig(px.bar(final_df, x="Poverty type", y="Poverty rate", animation_frame="PA amount", range_y=[-50, 0],
                 hover_data={"PA amount": False, "Poverty type": False}).update_layout(
    title_text='Poverty decrease at different levels of peronal allowance', 
    xaxis_title ="Poverty type",
    yaxis_title ="Poverty decrease",
    yaxis_ticksuffix="%",
    ).update_traces(marker_color='#1976D2'))

Part of the results above are explained by the fact that poor households tend to have more children and would therefore receive more UBI checks per household. Similarly, households in deep poverty tend to have fewer children than other poor households, which explains why the median gain is slightly less for this group.

In [269]:
#Mean income difference grouped by family type (not scrollable)
person_income_diff = sim_BI.calc("net_income") - sim.calc("net_income")
family = sim.calc("family_type", map_to = "person")

income_diff_by_family = person_income_diff.groupby(family).mean()

chart6 = format_fig(px.bar(x=["Couple without children", "Couple with children", "Single parent", "Single individual"], 
                           y=income_diff_by_family.round(),
                          labels={"x": "family type", "y":"change in income"}).update_layout(
    title_text='Average change in net income grouped by family type',
    yaxis_title="Change in net income",
    xaxis_title="Family type",
    yaxis_tickprefix="£"
))

In [765]:
#Mean income difference grouped by family type (scrollable)

family_type = pd.Series(["Couple without children", "Couple with children", "Single parent", "Single individual"])
df_list = []
sim_num = 0
def find_fam_diff(sim_BI): 
    person_income_diff = sim_BI.calc("net_income") - sim.calc("net_income")
    family = sim.calc("family_type", map_to = "person")
    income_diff_by_family = person_income_diff.groupby(family).mean().reset_index(drop = True)
    return income_diff_by_family.round()

for i in PA_amounts2:
    df = pd.DataFrame({"Family type": family_type, "Income change": find_fam_diff(sim_list[sim_num]), "PA amount": i})
    df_list.append(df)
    sim_num += 1


final_df = pd.concat(df_list)
final_df
format_fig(px.bar(final_df, x="Family type", y="Income change", animation_frame="PA amount", range_y=[-750,1100],
                 hover_data={"PA amount": False, "Family type": False}).update_layout(
    title_text='Average change in net income grouped by family type',
    yaxis_title="Change in net income",
    xaxis_title="",
    yaxis_tickprefix="£"
    ).update_traces(marker_color='#1976D2'))

In fact, this policy would benefit children more than any other group, as children do not lose income to a personal allowance decrease, and exclusively gain from the UBI amount given to them. Child poverty would fall 51%, compared to % for adult poverty. 

In [270]:
#Age poverty reductions (not scrollable)

#Child poverty reduction
poor_kids_before = sim.calc("in_poverty_bhc", map_to="person")[sim.calc("is_child")].sum()
poor_kids_after = sim_BI.calc("in_poverty_bhc", map_to="person")[sim_BI.calc("is_child")].sum()
child_pov_reduction = ((poor_kids_before - poor_kids_after)/poor_kids_before) * 100

#Percentage of poverty reduction for adults only
poor_adults_before = sim.calc("in_poverty_bhc", map_to="person")[sim.calc("is_adult")].sum()
poor_adults_after = sim_BI.calc("in_poverty_bhc", map_to="person")[sim_BI.calc("is_adult")].sum()
adult_pov_reduction = ((poor_adults_before - poor_adults_after)/poor_adults_before) * 100

#Overall
poverty_difference = sim_BI.calc("in_poverty_bhc", map_to="person").sum() - sim.calc("in_poverty_bhc", map_to="person").sum()
overall_reduction = (poverty_difference/sim.calc("in_poverty_bhc", map_to="person").sum()) * -100

chart5 = format_fig(px.bar(x=["Children", "Adults", "Overall"], y=[-child_pov_reduction.round(), -adult_pov_reduction.round(), -overall_reduction.round()],
                          labels={"x": "age group", "y":"poverty reduction"}).update_layout(
    title_text='Poverty changes by age group',
    yaxis_title="Reduction in poverty",
    xaxis_title="Age group",
    yaxis_ticksuffix="%"
))

In [763]:
#Age poverty reductions (scrollable)
age_group = pd.Series(["Children", "Adults", "Overall"])
df_list = []
sim_num = 0
def find_child_pov(sim_BI):
    poor_kids_before = sim.calc("in_poverty_bhc", map_to="person")[sim.calc("is_child")].sum()
    poor_kids_after = sim_BI.calc("in_poverty_bhc", map_to="person")[sim_BI.calc("is_child")].sum()
    child_pov_reduction = (((poor_kids_before - poor_kids_after)/poor_kids_before) * -100).round()
    return child_pov_reduction

def find_adult_pov(sim_BI):
    poor_adults_before = sim.calc("in_poverty_bhc", map_to="person")[sim.calc("is_adult")].sum()
    poor_adults_after = sim_BI.calc("in_poverty_bhc", map_to="person")[sim_BI.calc("is_adult")].sum()
    adult_pov_reduction = (((poor_adults_before - poor_adults_after)/poor_adults_before) * -100).round()
    return adult_pov_reduction

def find_overall_pov(sim_BI):
    poverty_difference = sim_BI.calc("in_poverty_bhc", map_to="person").sum() - sim.calc("in_poverty_bhc", map_to="person").sum()
    overall_reduction = ((poverty_difference/sim.calc("in_poverty_bhc", map_to="person").sum()) * 100).round()
    return overall_reduction

for i in PA_amounts2:
    changes = pd.Series([find_child_pov(sim_list[sim_num]), find_adult_pov(sim_list[sim_num]), find_overall_pov(sim_list[sim_num])])
    df = pd.DataFrame({"Age group": age_group, "Poverty change": changes, "PA amount": i})
    df_list.append(df)
    sim_num += 1


final_df = pd.concat(df_list)
#final_df
format_fig(px.bar(final_df, x="Age group", y="Poverty change", animation_frame="PA amount", range_y=[-60,0],
                 hover_data={"PA amount": False, "Age group": False}).update_layout(
    title_text='Poverty changes by age group',
    yaxis_title="Reduction in poverty",
    xaxis_title="",
    yaxis_ticksuffix="%"
    ).update_traces(marker_color='#1976D2'))

In [762]:
#Average children per household for each poverty group
isDeepPoor = sim.calc("in_deep_poverty_bhc")
isPoor = sim.calc("in_poverty_bhc") & ~isDeepPoor
isNotPoor = ~isPoor & ~isDeepPoor

poor_kids = sim.calc("num_children", map_to="household")[isPoor].mean()
deep_poor_kids = sim.calc("num_children", map_to="household")[isDeepPoor].mean()
not_poor_kids = sim.calc("num_children", map_to="household")[isNotPoor].mean()
format_fig(px.bar(x=["Deep poverty", "In poverty, but not deep", "Not in poverty"], y=[deep_poor_kids.round(2), poor_kids.round(2), not_poor_kids.round(2)],
                 labels={"y":"Average children", "x": "Household type"}).update_layout(
   #remove x label
    title_text="Average children per household for each poverty group",
    yaxis_title="Average number of children",
    xaxis_title="").update_traces(marker_color='#1976D2'))

As shown below, this would largely serve as an income transfer from adults to children.

In [426]:
#Mean difference in household income by age (not scrollable)
income_diff_2 = sim_BI.calc("household_net_income", map_to="person") - sim.calc("household_net_income", map_to="person")
age = sim.calc("age", map_to ="person")
chart4 = format_fig(px.bar(income_diff_2.groupby(age).mean().round(), labels={"index": "age", "value":"change in income"},
                          hover_data={"variable": False}).update_layout(
    title_text='Average change in household net income by age',
    yaxis_title="Change in household net income",
    yaxis_tickprefix="£",
    xaxis_title="Age",
    showlegend= False,
))

In [534]:
#Mean difference in household income by age (calc)
income_diff = sim_BI.calc("equiv_household_net_income") - sim.calc("equiv_household_net_income")
income = sim.calc("equiv_household_net_income")
age = sim.calc("age", map_to ="person")

ages = []
for i in range(81):
    ages.append(i)
ages = pd.Series(ages)

def find_income_diff2(sim_BI):
    diffs = sim_BI.calc("household_net_income", map_to="person") - sim.calc("household_net_income", map_to="person")
    return diffs.groupby(age).mean().round()

In [766]:
#Mean difference in household income by age (scrollable)
df_list = []
sim_num = 0
for i in PA_amounts2:
    df = pd.DataFrame({"Age": ages, "Income change": find_income_diff2(sim_list[sim_num]), "PA amount": i})
    df_list.append(df)
    sim_num += 1
    
final_df = pd.concat(df_list)
final_df
format_fig(px.bar(final_df, x="Age", y="Income change", animation_frame="PA amount", range_y=[-1100,3100],
                 hover_data={"PA amount": False}).update_layout(
    title_text='Average change in household net income by age',
    yaxis_title="Change in household net income",
    yaxis_tickprefix="£",
    xaxis_title="Age",
    showlegend= False,
    ).update_traces(marker_color='#1976D2'))

#### Replacing part of the personal allowance

Replacing the entire personal allowance with a UBI would reduce poverty and inequality, but what about only replacing part?

Our analysis shows each £2,000 decrease in the personal allowance could fund an increase in the UBI amount of around £110.

In [666]:
#UBI amount at different levels of PA (graph + calc)

def find_ubi_amount(PA_amount):
    
    def update_PA_parameter(parameters):
        parameters.tax.income_tax.allowances.personal_allowance.amount.update(period=periods.period("year:2020:1"), value=PA_amount)
        return parameters
    
    class reform(Reform):
        def apply(self):
            self.modify_parameters(update_PA_parameter)
    
    sim_less_PA = Microsimulation(reform, input_year=2020)
    revenue = sim.calc("net_income").sum()
    revenue_diff = revenue - sim_less_PA.calc("net_income").sum()
    BI_amount = (revenue_diff/(sim.calc("people").sum()))/52
    
    return BI_amount.round()

PA_amounts = [12500, 11500, 10500, 9500, 8500, 7500, 6500, 5500, 4500, 3500, 2500, 1500, 500, 0]
UBI_amounts = [find_ubi_amount(i) for i in (PA_amounts)]


PA_text = ["12500", "11500", "10500", "9500", "8500", "7500", "6500", "5500", "4500", "3500", "2500", "1500", "500", "0"]
chart = format_fig(px.line(x=PA_text, y=UBI_amounts, labels={"x": "Personal allowance amount", "y":"UBI amount"}).update_layout(
    title_text='Amount of UBI at different levels of personal allowance reduction',
    xaxis_title ="Amount of personal allowance (£)",
    yaxis_title ="UBI (£)",
))

In [756]:
#UBI amount at different levels of PA (just graph)
chart = format_fig(px.line(x=PA_amounts, y=UBI_amounts, labels={"x": "Personal allowance amount", "y":"UBI amount"}).update_layout(
    title_text='Amount of weekly UBI at different levels of personal allowance reduction',
    xaxis_title ="Reduction in personal allowance (£)",
    yaxis_title ="UBI (£)",
))

The following graph shows that funding UBI through an only partial decrease in personal allowance reduces the aforementioned effects on poverty. Each £2,000 of personal allowance reduced buys around a 5% decrease in poverty, and each £2,000 of personal allowance preserved decreases the poverty effect by 5 percentage points. 

In [614]:
#pov rates at different levels of PA (calculations)
def find_poverty_diff(sim_BI):
    poverty_difference = sim.calc("in_poverty_bhc", map_to="person").sum() - sim_BI.calc("in_poverty_bhc", map_to="person").sum()
    percentage_difference = (poverty_difference/sim.calc("in_poverty_bhc", map_to="person").sum()) * 100
    return -percentage_difference

PA_amounts = [12500, 11500, 10500, 9500, 8500, 7500, 6500, 5500, 4500, 3500, 2500, 1500, 500, 0]

sim_num = 0
pov_diffs = []
for i in PA_amounts2:
    diff = find_poverty_diff(sim_list[sim_num]).round()
    pov_diffs.append(diff)
    sim_num += 1

df = pd.DataFrame({"PA amounts": pd.Series(PA_amounts2), "Poverty difference": pd.Series(pov_diffs), "UBI amount": pd.Series(UBI_amounts_text)})

def find_deep_poverty_diff(sim_BI):
    deep_poverty_difference = sim.calc("in_deep_poverty_bhc", map_to="person").sum() - sim_BI.calc("in_deep_poverty_bhc", map_to="person").sum()
    percentage_difference = (deep_poverty_difference/sim.calc("in_deep_poverty_bhc", map_to="person").sum()) * 100
    return -percentage_difference

sim_num = 0
deep_pov_diffs = []
for i in PA_amounts2:
    diff = find_deep_poverty_diff(sim_list[sim_num]).round()
    deep_pov_diffs.append(diff)
    sim_num += 1

df2 = pd.DataFrame({"PA amounts": pd.Series(PA_amounts2), "Deep poverty difference": pd.Series(deep_pov_diffs), "UBI amount": pd.Series(UBI_amounts_text)})


In [752]:
UBI_amounts_text=["UBI=0", "UBI=2", "UBI=4", "UBI=6", "UBI=8", "UBI=10", "UBI=12", "UBI=15", "UBI=17", "UBI=20", "UBI=22", "UBI=25", "UBI=27",
                 "UBI=29"]

In [857]:
#pov rates at different levels of PA (graph)

df = pd.DataFrame({"PA amounts": pd.Series(PA_amounts2), "Poverty difference": pd.Series(pov_diffs), "UBI amount": pd.Series(UBI_amounts_text)})

chart8 = format_fig(go.Figure(data=[
    go.Scatter(name='Poverty', x=df["PA amounts"], y=df["Poverty difference"], hovertext=df["UBI amount"], marker_color='#49A6E2', mode='lines'),
    go.Scatter(name='Deep poverty', x=df2["PA amounts"], y=df2["Deep poverty difference"], hovertext=df2["UBI amount"], marker_color="#0F4AA1", mode='lines'),
]).update_layout( 
    title_text='Poverty decreases at different levels of personal allowance', 
    xaxis_title ="Personal allowance amount",
    yaxis_title ="Decrease in poverty",
    yaxis_ticksuffix="%"))


#### Conclusion

Replacing personal allowance with a budget-neutral UBI is an effective mechanism for cutting poverty. In the Liberal Democrats’ explored UBI policies, eliminating it fully could reduce poverty by over an additional 5% (additional suggestions can be found in our [report](https://www.ubicenter.org/progressive-adjustments-lib-dem-working-group) on progressive adjustments for the LibDem's UBI reform). 

Furthermore, a disproportionate amount of this benefit would go to children, and since child poverty has well-documented impacts on [educational](https://www.jrf.org.uk/sites/default/files/jrf/migrated/files/2123.pdf) and [health](https://adc.bmj.com/content/archdischild/101/8/759.full.pdf) outcomes, this has the potential to be a high-impact policy for society as a whole.

Given these findings, UBI proponents in the UK should take a serious look at whether keeping the personal allowance is worthwhile, provided the benefits of the UBI that could be funded through taxing it.

In [904]:
#gini graph
sim_num = 0
ginis = []
for i in PA_amounts2:
    gini = sim_list[sim_num].calc("household_net_income", map_to="person").gini()
    ginis.append(gini)
    sim_num += 1
ginis

gini2 = pd.Series(ginis)
chart1 = format_fig((px.line(x=PA_amounts2, y=gini2).update_layout(
    title_text='Gini coefficient at different levels of PA',
    xaxis_title ="Level of PA",
    yaxis_title ="Gini coefficient",
    showlegend= False,
    yaxis_ticksuffix="%"
).update_traces(marker_color='#1976D2')))
chart1

In [642]:
#Income % change for deciles w vs w/o PA
class PA_reform(Reform):
    def apply(self):
        self.neutralize_variable("personal_allowance")
        
sim_no_PA = Microsimulation(PA_reform)

In [903]:
income_with_PA = sim.calc("equiv_household_net_income") 
income_no_PA = sim_no_PA.calc("equiv_household_net_income")
decile_change = []
for n in range (1, 11):
    #total income difference before vs after for the decile
    id = income_with_PA.groupby(income.decile_rank()).sum()[n] - income_no_PA.groupby(income.decile_rank()).sum()[n]
    #average income difference for the decile
    id2 = id/income_with_PA.groupby(income.decile_rank()).count()[n]
    #average income difference of the decile as a percent of average income for the decile
    id3 = id2/income_with_PA.groupby(income.decile_rank()).mean()[n] * 100
    decile_change.append(id3)

change = pd.Series(decile_change)
chart1 = format_fig((px.bar(y=decile_change).update_layout(
    title_text='Income increase thanks to PA for households in each income decile',
    xaxis_title ="Household income decile",
    yaxis_title ="Change in household net income",
    showlegend= False,
    yaxis_ticksuffix="%"
).update_traces(marker_color='#1976D2')))
chart1

# New graphs

In [907]:
from openfisca_uk import IndividualSim
#mtr for single person
#how to add reform
sim_ind = IndividualSim()
sim_ind.add_person(name="adult", age=40, is_benunit_head=True, is_household_head=True)
sim_ind.add_benunit(adults=["adult"])
sim_ind.add_household(adults=["adult"])

sim_ind.vary("employment_income", min=0, max=200000, step=100)

earnings = sim_ind.calc("employment_income", target="adult")
mtr = sim_ind.calc_mtr(wrt_target="adult")

fig = px.line(x=earnings, y=mtr, title="Effective marginal tax schedule")
fig.update_layout(yaxis_tickformat="%", xaxis_tickprefix="£", xaxis_title="Earnings", yaxis_title="Marginal tax rate")
fig.show()

In [908]:
#mtr for couple with children
sim2 = IndividualSim()
sim2.add_person(name="parent", age=24, is_benunit_head=True, is_household_head=True)
sim2.add_person(name="parent2", age=34, is_benunit_head=True, is_household_head=True)
sim2.add_person(name="child", age=2)
sim2.add_person(name="child2", age=10)
sim2.add_benunit(adults=["parent", "parent2"], children=["child", "child2"], claims_child_benefit=False)
sim2.add_household(adults=["parent, parent2"], children=["child", "child2"])

sim2.vary("employment_income", min=0, max=200000, step=100)

earnings = sim2.calc("employment_income", target="parent")
mtr = sim2.calc_mtr(wrt_target="parent")

fig = px.line(x=earnings, y=mtr, title="Effective marginal tax schedule")
fig.update_layout(yaxis_tickformat="%", xaxis_tickprefix="£", xaxis_title="Earnings", yaxis_title="Marginal tax rate")
fig.show()

SituationParsingError: {'households': {'household': {'adults': 'Unexpected value: parent, parent2. parent, parent2 has been declared in household adults, but has not been declared in people.'}}}

In [812]:
#employment income and net income chart for single person

from openfisca_uk import IndividualSim
from ubicenter import format_fig
GERY = "#BDBDBD"
BLUE = "#1976D2" 
individual_colors = [GREY, BLUE]
def plot_budget(household_config, title):
    baseline_policy = IndividualSim()
    household_config(baseline_policy)
    baseline_policy.vary("employment_income")
    employment_income = baseline_policy.calc("employment_income")[0]
    budget_dfs = []
    for reform, name in zip(
        sim_list, PA_amounts2
    ):
        ubi_policy = IndividualSim(reform)
        household_config(ubi_policy)
        ubi_policy.vary("employment_income")
        df = pd.DataFrame(
            {
                "Net income (Baseline)": baseline_policy.calc(
                    "household_net_income"
                )[0],
                "Net income (Reform)": ubi_policy.calc("household_net_income")[
                    0
                ],
                "Tax (Baseline)": baseline_policy.calc("tax")[0],
                "Tax (Reform)": ubi_policy.calc("tax")[0],
                "Benefits (Baseline)": baseline_policy.calc("benefits")[0],
                "Benefits (Reform)": ubi_policy.calc("benefits")[0],
                "UBI (Reform)": ubi_policy.calc("UBI")[0],
                "Employment income": employment_income,
                "MTR (Baseline)": 1
                - baseline_policy.calc_deriv(
                    "household_net_income",
                    var_target="household",
                    wrt_target="adult",
                ),
                "MTR (Reform)": 1
                - ubi_policy.calc_deriv(
                    "household_net_income",
                    var_target="household",
                    wrt_target="adult",
                ),
                "Tax MTR (Baseline)": baseline_policy.calc_deriv(
                    "tax", var_target="adult", wrt_target="adult"
                ),
                "Tax MTR (Reform)": ubi_policy.calc_deriv(
                    "tax", var_target="adult", wrt_target="adult"
                ),
                "Income Tax MTR (Baseline)": baseline_policy.calc_deriv(
                    "income_tax", var_target="adult", wrt_target="adult"
                ),
                "Income Tax MTR (Reform)": ubi_policy.calc_deriv(
                    "income_tax", var_target="adult", wrt_target="adult"
                ),
                "NI MTR (Baseline)": baseline_policy.calc_deriv(
                    "national_insurance",
                    var_target="adult",
                    wrt_target="adult",
                ),
                "NI MTR (Reform)": ubi_policy.calc_deriv(
                    "national_insurance",
                    var_target="adult",
                    wrt_target="adult",
                ),
                "Benefit MTR (Baseline)": -baseline_policy.calc_deriv(
                    "benefits", var_target="adult", wrt_target="adult"
                ),
                "Benefit MTR (Reform)": -ubi_policy.calc_deriv(
                    "benefits", var_target="adult", wrt_target="adult"
                ),
            }
        )
        df["UBI"] = f"£{name}/week"
        budget_dfs += [df]

    fig = px.line(
        pd.concat(budget_dfs),
        x="Employment income",
        y=[
            "Net income (Baseline)",
            "Net income (Reform)",
            "Tax (Baseline)",
            "Tax (Reform)",
            "Benefits (Baseline)",
            "Benefits (Reform)",
            "UBI (Reform)",
        ],
        animation_frame="UBI",
        color_discrete_sequence=individual_colors,
    ).update_layout(
        title=title,
        yaxis_tickprefix="£",
        xaxis_tickprefix="£",
        yaxis_title="Yearly amount",
        xaxis_title="Employment income",
        legend_title_text="",
    )
    hidden = [False] * 2 + [True] * 5
    for i in range(7):
        if hidden[i]:
            fig.data[i].visible = "legendonly"

    return format_fig(fig), pd.concat(budget_dfs)


def single_person_UC(sim):
    sim.add_person(age=26, is_benunit_head=True, name="adult"),
    sim.add_benunit(adults=["adult"], claims_UC=True),
    sim.add_household(adults=["adult"])


fig, budget_df = plot_budget(
    single_person_UC,
    "Effect of UBI reforms on the relationship between employment income and net income",
)
fig

TypeError: 'Microsimulation' object is not callable

In [868]:
#within decile

NAMES = (
        "Gain more than 5%",
        "Gain less than 5%",
        "No change",
        "Lose less than 5%",
        "Lose more than 5%"
    )

def intra_decile_graph_data(baseline, *reform_sims):
    AMOUNTS = PA_amounts2
    l = []
    for i, reform_sim in enumerate(reform_sims):
        income = baseline.calc("equiv_household_net_income", map_to="person")
        decile = income.decile_rank()
        gain = reform_sim.calc("household_net_income", map_to="person") - baseline.calc("household_net_income", map_to="person")
        rel_gain = (gain / baseline.calc("household_net_income", map_to="person")).dropna()
        bands = (None, 0.05, 1e-3, -1e-3, -0.05, None)
        for upper, lower, name in zip(bands[:-1], bands[1:], NAMES):
            fractions = []
            for j in range(1, 11):
                subset = rel_gain[decile == j]
                if lower is not None:
                    subset = subset[rel_gain > lower]
                if upper is not None:
                    subset = subset[rel_gain <= upper]
                fractions += [subset.count() / rel_gain[decile == j].count()]
            tmp = pd.DataFrame({"Personal allowance": f"£{AMOUNTS[i]}/week", "fraction": fractions, "decile": list(range(1, 11)), "Outcome": name})
            l.append(tmp)
    return pd.concat(l).reset_index()